In [1]:
import pandas as pd
import numpy as np
import urllib
import pyodbc
import pandas as pd
import sqlalchemy as sa
import numpy as np
import math

connection_string = (
    'Driver={ODBC Driver 17 for SQL Server};'
    'SERVER=118.69.201.34;'
    'Database=FLC_SHOP_SALES_DATAWAREHOUSE;'
    'UID=ecom_user;'
    'PWD=Ec0m@12345;'
    'Trusted_Connection=no;'
)

connection_uri = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(str(connection_string))}"
engine = sa.create_engine(connection_uri, fast_executemany=True)

In [2]:
df_shop = pd.read_csv(r'D:\CHIPHISHOP\VD_CID_SHOP_LC.csv')
df_shop

,SHOP_KEY,REGION_NAME_2
0,7940,Unspecified
1,8029,Unspecified
2,2513,Unspecified
3,789,Miền Tây
4,202,Hồ Chí Minh
...,...,...
1183,384,Miền Tây
1184,412,Miền Tây
1185,161,Miền Tây
1186,273,Miền Đông


In [3]:
df_segment = pd.read_csv(r'D:\MANTINH\SEGMENT.csv', header = None)
df_segment.rename(columns={df_segment.columns[0]: 'CUSTOMER_PHONE', df_segment.columns[1]: 'SEGMENT'}, inplace = True)
df_segment

,CUSTOMER_PHONE,SEGMENT
0,82B70FEAC2077833F9FBF651692B530D,lost
1,D1C9FF6681AA051C794FCC3A88DC52E2,lost
2,B08B476C30612A33330B207C74CCEE63,lost
3,2B16DE9F414B57B748010811706D4558,prom
4,494499E83CD373469863034377C768E9,calo
...,...,...
12716661,89D3595BD085CEBA3E44638C05BE0EAD,new_cus
12716662,8EE60161A6C83ED41E80F89D02C27079,new_cus
12716663,475F3536B2CF3371A10371C2568EDC07,recent
12716664,637DC286F33C7B1ED646F830ACF22793,new_cus


In [4]:
remove_lost = df_segment.loc[df_segment['SEGMENT'] != 'lost']
remove_lost

,CUSTOMER_PHONE,SEGMENT
3,2B16DE9F414B57B748010811706D4558,prom
4,494499E83CD373469863034377C768E9,calo
5,09E0B80E7081060D858A3F093BDD3B35,hib
6,447601DF53B4A54EB9BB16A042EA7AEF,recent
8,5FA366BAFCBE81B83AABFC6D03380C04,recent
...,...,...
12716661,89D3595BD085CEBA3E44638C05BE0EAD,new_cus
12716662,8EE60161A6C83ED41E80F89D02C27079,new_cus
12716663,475F3536B2CF3371A10371C2568EDC07,recent
12716664,637DC286F33C7B1ED646F830ACF22793,new_cus


In [4]:
df_sku_mantinh = pd.read_excel(r'D:\MANTINH\sku_thuoc_man_tinh_final.xlsx')
df_sku_mantinh_drop = df_sku_mantinh.drop(df_sku_mantinh.columns[[1,2,3,4,6]], axis=1).drop_duplicates()
df_sku_mantinh_drop.rename(columns={df_sku_mantinh_drop.columns[0]: 'PRODUCT_CODE', df_sku_mantinh_drop.columns[1]: 'TYPE'}, inplace = True)
df_sku_mantinh_drop

,PRODUCT_CODE,TYPE
0,22389,CÒN LẠI
1,21253,CÒN LẠI
2,548,KHÁCH ĐẶT MỚI LẤY
3,900,CÒN LẠI
4,28633,CÒN LẠI
...,...,...
2169,6307,CÒN LẠI
2170,7853,HẾT HÀNG
2171,1403,CÒN LẠI
2172,29349,CÒN LẠI


In [5]:
df_mantinh = pd.read_csv(r'D:\MANTINH\CUSTOMER_CHRONIC_BILL.csv', header = None)
df_mantinh.rename(columns={df_mantinh.columns[0]: 'DATE_KEY', df_mantinh.columns[1]: 'SHOP_KEY', df_mantinh.columns[2]: 'CUSTOMER_PHONE', df_mantinh.columns[3]: 'PRODUCT_CODE',df_mantinh.columns[4]: 'AMT', df_mantinh.columns[5]: 'C_BILL'}, inplace = True)
# df_mantinh = df_mantinh.drop(df_mantinh.columns[[2]], axis=1).drop_duplicates()
print(df_mantinh['CUSTOMER_PHONE'].nunique())
df_mantinh

4455278


,DATE_KEY,SHOP_KEY,CUSTOMER_PHONE,PRODUCT_CODE,AMT,C_BILL
0,20220104,1060,7B21A4516A0B2BE338D0896D9ED0CB4E,18384,197010.0,1
1,20220412,1042,6C8CB0168F331ABF8179E10E5FB73BC0,20370,117000.0,1
2,20200511,1143,B30544BDC5C0540C2B8C341C468A7B6C,20694,98000.0,1
3,20221020,502,1AE26D22005E813B114F4DD3DFDBE299,5750,666400.0,1
4,20200304,1424,781E5E245D69B566979B86E28D23F2C7,7339,284812.0,2
...,...,...,...,...,...,...
29946565,20211229,1174,DBCF11645DB7360EECF67471FF5A5AD0,5272,164500.0,1
29946566,20220312,1330,60482335488B9732F7078D42CB325ADD,8304,220000.0,1
29946567,20221123,1770,FDFAC9AF041821EC19BA1D790EC007E2,2063,96000.0,1
29946568,20220626,2,079FEC9EB9C25912EF5E4FF468C050D0,3431,48300.0,1


In [6]:
filter_mantinh_2022 = df_mantinh.loc[df_mantinh['DATE_KEY'] >= 20170101]
filter_mantinh_2022 = filter_mantinh_2022.drop(filter_mantinh_2022.columns[[0,4,5]], axis=1).drop_duplicates()
print(filter_mantinh_2022['CUSTOMER_PHONE'].nunique())
filter_mantinh_2022

4455278


,SHOP_KEY,CUSTOMER_PHONE,PRODUCT_CODE
0,1060,7B21A4516A0B2BE338D0896D9ED0CB4E,18384
1,1042,6C8CB0168F331ABF8179E10E5FB73BC0,20370
2,1143,B30544BDC5C0540C2B8C341C468A7B6C,20694
3,502,1AE26D22005E813B114F4DD3DFDBE299,5750
4,1424,781E5E245D69B566979B86E28D23F2C7,7339
...,...,...,...
29946549,581,CD3D53D1744F1147D897CFE5352FD1B5,28627
29946553,684,F342F616FDA772C440FA906BDFCF37FD,3239
29946561,1061,9AF0CF2FC283D19CFC1777419D85CEC0,10976
29946564,1837,74141CB73683A7C0D18A80FE35A21074,5532


In [17]:
df_merge_mantinh = df_segment.merge(filter_mantinh_2022, how='right', on='CUSTOMER_PHONE') #Merge khách hàng mãn tính và segment
# df_merge_mantinh = df_merge_mantinh.drop(df_merge_mantinh.columns[[1,3]], axis=1).drop_duplicates()
df_merge_shop_mantinh = df_shop.merge(df_merge_mantinh, how='inner', on='SHOP_KEY')
df_merge_mantinh_all = df_merge_shop_mantinh.merge(df_sku_mantinh_drop, how='inner', on='PRODUCT_CODE').fillna('Khác')
df_merge_mantinh_all = df_merge_mantinh_all.drop(df_merge_mantinh_all.columns[[0,4]], axis=1).drop_duplicates()
print(df_merge_mantinh_all['CUSTOMER_PHONE'].nunique())
df_merge_mantinh_all

3503030


,REGION_NAME_2,CUSTOMER_PHONE,SEGMENT,TYPE
0,Miền Tây,BB1C55F65D6B9DE84B1FDA7813E6B04B,ats,HÀNG KHAN HIẾM
1,Miền Tây,4300A8581FBA2C29BADBE0EAB20485D7,ats,HÀNG KHAN HIẾM
2,Miền Tây,FAB1F766376EF6E70E92BEBE2B198F6B,prom,HÀNG KHAN HIẾM
3,Miền Tây,96FD10F2644088F19307A1FD75242C47,ats,HÀNG KHAN HIẾM
4,Miền Tây,AB94DFBD31C601FA38D738E3EDB32349,recent,HÀNG KHAN HIẾM
...,...,...,...,...
10203827,Miền Trung,A5898B3C0F621023CAE043F8072FE73B,champ,NGỪNG ĐẶT HÀNG
10203828,Hồ Chí Minh,35FC1CA3DCA807D888E1147E0A309B60,loyal,KHÁCH ĐẶT MỚI LẤY
10203829,Miền Tây,8410B28C78C8E6A10EACF5EC7AC98270,champ,KHÁCH ĐẶT MỚI LẤY
10203830,Miền Tây,D25A9C313E2CA039B7D3EFB99D0FDE34,calo,DATE CẬN


In [18]:
df_merge_mantinh_all.loc[df_merge_mantinh_all['SEGMENT'] == 'Khác']

,REGION_NAME_2,CUSTOMER_PHONE,SEGMENT,TYPE
7,Miền Tây,05FB7031D9B85556E40DEB0CAD76C7DB,Khác,HÀNG KHAN HIẾM
13,Miền Tây,A9A0556A79904FA410D4990F4479E103,Khác,HÀNG KHAN HIẾM
62,Miền Tây,E1B73104AD01BE8759CF50C2A2F16749,Khác,HÀNG KHAN HIẾM
65,Miền Tây,9F21FBCE29E612978661614B1571C44D,Khác,HÀNG KHAN HIẾM
67,Miền Tây,120E961A1C2E860B96946A327C085D60,Khác,HÀNG KHAN HIẾM
...,...,...,...,...
10203632,Hồ Chí Minh,D72DABD355AC5B392EA3B7379AE9F0EF,Khác,HÀNG MỞ TỦ
10203684,Hà Nội,6565D1C3A1803E73EA724DAAAFEF7CD2,Khác,NGỪNG ĐẶT HÀNG
10203691,Hà Nội,E9ACB793B4A9F84A19189C5065281D9A,Khác,KHÁCH ĐẶT MỚI LẤY
10203801,Hồ Chí Minh,35D287F9604DC4A362379543CEFB8E3A,Khác,NGỪNG ĐẶT HÀNG


In [ ]:
df_cus_all = pd.read_csv(r'D:\CHIPHISHOP\CUSTOMER_ALL.csv', header = None)
df_cus_all.rename(columns={df_cus_all.columns[0]: 'CUSTOMER_PHONE', df_cus_all.columns[1]: 'SHOP_KEY'}, inplace = True)
df_merge_shop_all = df_shop.merge(df_cus_all, how='inner', on='SHOP_KEY')
df_merge_all = df_segment.merge(df_merge_shop_all, how='right', on='CUSTOMER_PHONE')
df_merge_all
# df_merge_all = df_merge_all.drop(df_merge_all.columns[[0,3]], axis=1).drop_duplicates()
print(df_merge_all['CUSTOMER_PHONE'].nunique())
df_merge_all

In [ ]:
filter_DATE_CAN = df_khac.loc[df_khac['TYPE'] == 'HÀNG KHAN HIẾM']
# filter_DATE_CAN = filter_DATE_CAN.drop(filter_DATE_CAN.columns[[0,3]], axis=1).drop_duplicates()
filter_DATE_CAN['CUSTOMER_PHONE'].nunique()
filter_DATE_CAN

In [ ]:
df_groupby_mantinh_DATE_CAN = filter_DATE_CAN.groupby(['REGION_NAME_2']).count().reset_index()
df_groupby_mantinh_DATE_CAN

In [ ]:
df_pivot_mantinh_DATE_CAN = df_groupby_mantinh_DATE_CAN.pivot_table(values='CUSTOMER_PHONE', index='REGION_NAME_2', columns='SEGMENT',)
df_pivot_mantinh_DATE_CAN

In [ ]:
df_pivot_mantinh_DATE_CAN.to_excel(r'D:\CHIPHISHOP\CONLAI.xlsx')

In [ ]:
df_khac_1 = df_khac.drop(df_khac.columns[2], axis=1).drop_duplicates()
df_all_khac = df_khac_1.groupby(['REGION_NAME_2']).count().reset_index()
df_all_khac
df_all_khac.to_excel(r'D:\CHIPHISHOP\CONLAI.xlsx')

In [253]:
filter_ = df_merge_all.loc[df_merge_all['SEGMENT'].isnull()]
filter_

In [167]:
df_merge_cus_mantinh = df_merge_all.drop(df_merge_all.columns[[1]], axis=1).drop_duplicates()
df_merge_cus_mantinh

In [ ]:
df_merge_cus_mantinh_groupby = df_merge_cus_mantinh.groupby(['SEGMENT']).count().reset_index()
df_merge_cus_mantinh_groupby

In [36]:
df_merge_cus_mantinh_groupby.to_excel(r'D:\CHIPHISHOP\MANTINH_SEGMENT.xl')

In [23]:
df_merge_all

,SHOP_KEY,REGION_NAME_2,CUSTOMER_PHONE
0,789,Miền Tây,141E9DB3F102BFE4C36760304D9A2856
1,789,Miền Tây,C01700FA7CB7D50D6834ABADBB1A6A9A
2,789,Miền Tây,20DCCB794EF8225391F0C3E18F027747
3,789,Miền Tây,A2173107FF3491D31EC274D8B5CA4B12
4,789,Miền Tây,11DFA97979E4BBE8F8090806B55E29FB
...,...,...,...
492538,11040,Miền Đông,39D4ACF00C7758547387246EAC01E58F
492539,11040,Miền Đông,AC0DB0F5E6711A80B2AF3A916344C5FC
492540,11040,Miền Đông,FF09136ACF8D22328447DCC7D2FCC7E4
492541,11040,Miền Đông,5F277877013484FE8E57376DD66B7E1E


In [24]:
df_groupby_mantinh = df_merge_all.groupby(['REGION_NAME_2']).count().reset_index()
df_groupby_mantinh

In [41]:
df_pivot_mantinh = df_groupby_mantinh.pivot_table(values='CUSTOMER_PHONE', index='REGION_NAME_2', columns='SEGMENT',)
df_pivot_mantinh
# df_pivot_mantinh.to_excel(r'D:\CHIPHISHOP\MANTINH1.xlsx')